In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/taoliu/Data/NAS-Projects-master/lib'))
sys.path.append(str('/home/taoliu/Data/NAS-Projects-master/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

/data/taoliu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument('--data_path',          type=str,   default= '/data/taoliu/ENNAS_benchmark102/data/cifar-10-batches-py',    help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10',choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102', help='The search space name.')
parser.add_argument('--config_path',        type=str,   default= '/home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/algos/RANDOM.config'  ,help='The path to the configuration.')
parser.add_argument('--max_nodes',          type=int,   default= 4 , help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16 , help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
parser.add_argument('--select_num',         type=int,   default= 100, help='The number of selected architectures to evaluate.')
# log
parser.add_argument('--workers',            type=int,   default=2, help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='./RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/taoliu/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=200,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=0,help='manual seed')
args = parser.parse_args([])

In [3]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    arch_pre,genotypes_pre,operations_pre=network.module.random_genotype( False )
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)
    
        # update the weights
        arch,genotypes,operations=network.module.random_genotype( True )
        
                
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        loss = criterion(logits, base_targets)
               
        
        arch_pre=network.module.set_genotype(arch_pre)
        _, logits = network(base_inputs)
        pre_loss = criterion(logits, base_targets)  
        lbd=0.2
        arch=network.module.set_genotype(arch)
        base_loss=(1-lbd)*loss+lbd*(pre_loss)
        
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
            # record
        base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
        base_losses.update(base_loss.item(),  base_inputs.size(0))
        base_top1.update  (base_prec1.item(), base_inputs.size(0))
        base_top5.update  (base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        
        #####current to pre
        arch_pre,genotypes_pre,operations_pre=arch,genotypes,operations
        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = '*SEARCH* ' + time_string() + ' [{:}][{:03d}/{:03d}]'.format(epoch_str, step, len(xloader))
            Tstr = 'Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})'.format(batch_time=batch_time, data_time=data_time)
            Wstr = 'Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=base_losses, top1=base_top1, top5=base_top5)
            logger.log(Sstr + ' ' + Tstr + ' ' + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg


def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
          # measure data loading time
            data_time.update(time.time() - end)
       # prediction

            network.module.random_genotype( True )
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
      # record
            arch_prec1, arch_prec5 = obtain_accuracy(logits.data, arch_targets.data, topk=(1, 5))
            arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
            arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))
      # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

def search_find_best(valid_loader, network, criterion, select_num):
    best_acc = 0
    for iarch in range(select_num):
        arch, genotypes,operations = network.module.random_genotype( True )
        valid_a_loss, valid_a_top1, valid_a_top5  = valid_func(valid_loader, network, criterion)
        if best_acc < valid_a_top1:
            best_arch, best_acc = arch, valid_a_top1
    return best_arch

In [4]:
#def main(xargs):
xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
#elif xargs.dataset.startswith('ImageNet16'):
#  # all_indexes = list(range(len(train_data))) ; random.seed(111) ; random.shuffle(all_indexes)
#  # train_split, valid_split = sorted(all_indexes[: len(train_data)//2]), sorted(all_indexes[len(train_data)//2 :])
#  # imagenet16_split = dict2config({'train': train_split, 'valid': valid_split}, None)
#  # _ = configure2str(imagenet16_split, 'temp.txt')
#  split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
#  imagenet16_split = load_config(split_Fpath, None, None)
#  train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
#  logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
config = load_config(xargs.config_path, {'class_num': class_num, 'xshape': xshape}, logger)
logger.log('config : {:}'.format(config))
# To split data
train_data_v2 = deepcopy(train_data)
train_data_v2.transform = valid_data.transform
valid_data    = train_data_v2
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
valid_loader  = torch.utils.data.DataLoader(valid_data, batch_size=config.batch_size, sampler=torch.utils.data.sampler.SubsetRandomSampler(valid_split), num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'RANDOM', 'C': xargs.channel, 'N': xargs.num_cells,
                          'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                          'space'    : search_space}, None)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)
logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
if xargs.arch_nas_dataset is None: api = None
else                             : api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists(): # automatically resume from previous checkpoint
    logger.log("=> loading checkpoint of the last-info '{:}' start".format(last_info))
    last_info   = torch.load(last_info)
    start_epoch = last_info['epoch']
    checkpoint  = torch.load(last_info['last_checkpoint'])
    genotypes   = checkpoint['genotypes']
    valid_accuracies = checkpoint['valid_accuracies']
    search_model.load_state_dict( checkpoint['search_model'] )
    w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
    w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
    logger.log("=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(last_info, start_epoch))
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {'best': -1}, {}

# start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
    epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
    logger.log('\n[Search the {:}-th epoch] {:}, LR={:}'.format(epoch_str, need_time, min(w_scheduler.get_lr())))

# selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, epoch_str, xargs.print_freq, logger)
    search_time.update(time.time() - start_time)
    logger.log('[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s'.format(epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum))
    valid_a_loss , valid_a_top1 , valid_a_top5  = valid_func(valid_loader, network, criterion)
    logger.log('[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%'.format(epoch_str, valid_a_loss, valid_a_top1, valid_a_top5))
    #cur_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    cur_arch = search_find_best(valid_loader, network, criterion, 2)
    genotypes[epoch] = cur_arch
# check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies['best']:
        valid_accuracies['best'] = valid_a_top1
        find_best = True
    else: find_best = False

# save checkpoint
    save_path = save_checkpoint({'epoch' : epoch + 1,
            'args'  : deepcopy(xargs),
            'search_model': search_model.state_dict(),
            'w_optimizer' : w_optimizer.state_dict(),
            'w_scheduler' : w_scheduler.state_dict(),
            'genotypes'   : genotypes,
            'valid_accuracies' : valid_accuracies},
            model_base_path, logger)
    last_info = save_checkpoint({
      'epoch': epoch + 1,
      'args' : deepcopy(args),
      'last_checkpoint': save_path,
      }, logger.path('info'), logger)
    if find_best:
        logger.log('<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.'.format(epoch_str, valid_a_top1))
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None: logger.log('{:}'.format(api.query_by_arch( genotypes[epoch] )))
# measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log('\n' + '-'*200)
logger.log('Pre-searching costs {:.1f} s'.format(search_time.sum))
start_time = time.time()
best_arch, best_acc = None, -1
for iarch in range(xargs.select_num):
    arch = search_model.random_genotype( True )
    valid_a_loss, valid_a_top1, valid_a_top5  = valid_func(valid_loader, network, criterion)
    logger.log('final evaluation [{:02d}/{:02d}] : {:} : accuracy={:.2f}%, loss={:.3f}'.format(iarch, xargs.select_num, arch, valid_a_top1, valid_a_loss))
    if best_arch is None or best_acc < valid_a_top1:
        best_arch, best_acc = arch, valid_a_top1
search_time.update(time.time() - start_time)
logger.log('RANDOM-NAS finds the best one : {:} with accuracy={:.2f}%, with {:.1f} s.'.format(best_arch, best_acc, search_time.sum))
if api is not None: logger.log('{:}'.format( api.query_by_arch(best_arch[0]) ))
logger.close()

Main Function with logger : Logger(dir=RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : /home/taoliu/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth
channel          : 16
config_path      : /home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/algos/RANDOM.config
data_path        : /data/taoliu/ENNAS_benchmark102/data/cifar-10-batches-py
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 0
save_dir         : ./RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
select_num       : 100
workers          : 2
Python  Version  : 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56)  [GCC 7.2.0]
Pillow  Version  : 5.1.0
PyTorch Version  : 0.4.1
cuDNN   Version  : 7102
CUDA available   : True
CUDA GPU numbers : 1
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verif

save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth
|nor_conv_1x1~0|+|nor_conv_1x1~0|avg_pool_3x3~1|+|nor_conv_3x3~0|nor_conv_1x1~1|none~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=1245
cifar10-valid  FLOP= 54.97 M, Params=0.400 MB, latency=19.92 ms.
cifar10-valid  train : [loss = 0.006, top1 = 99.93%], valid : [loss = 0.500, top1 = 88.94%]
cifar10        FLOP= 54.97 M, Params=0.400 MB, latency=21.00 ms.
cifar10        train : [loss = 0.010, top1 = 99.83%], test  : [loss = 0.315, top1 = 92.26%]
cifar100       FLOP= 54.97 M, Params=0.406 MB, latency=19.88 ms.
cifar100       train : [loss = 0.248, top1 = 93.86%], valid : [loss = 1.239, top1 = 68.62%], test : [loss = 1.262, top1 = 68.33%]
ImageNet16-120 FLOP= 13.75 M, Params=0.407 MB, latency=16.60 ms.
ImageNet16-120 train : [loss = 1.916, top1 = 49.51%], valid : [loss = 2.282, top1 = 42.23%], test : [loss = 2.289, top1 = 41.99%]

[Se

[006-250] evaluate  : loss=73.29, accuracy@1=11.52%, accuracy@5=54.43%
Find RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth
Find RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth exist, delete is at first before saving
save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth
|none~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|avg_pool_3x3~0|avg_pool_3x3~1|none~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=2822
cifar10-valid  FLOP= 47.10 M, Params=0.344 MB, latency=15.45 ms.
cifar10-valid  train : [loss = 0.548, top1 = 80.87%], valid : [loss = 0.865, top1 = 71.36%]
cifar10        FLOP= 47.10 M, Params=0.344 MB, latency=15.87 ms.
cifar10        train : [loss = 0.563, top1 = 80.4

[010-250] evaluate  : loss=217.45, accuracy@1=11.68%, accuracy@5=53.80%
Find RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth
Find RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth exist, delete is at first before saving
save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth
|none~0|+|nor_conv_3x3~0|skip_connect~1|+|nor_conv_1x1~0|nor_conv_3x3~1|skip_connect~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=4678
cifar10-valid  FLOP= 82.49 M, Params=0.587 MB, latency=17.78 ms.
cifar10-valid  train : [loss = 0.007, top1 = 99.94%], valid : [loss = 0.477, top1 = 88.96%]
cifar10        FLOP= 82.49 M, Params=0.587 MB, latency=17.28 ms.
cifar10        train : [loss = 0.012, to

[014-250] evaluate  : loss=24.61, accuracy@1=11.04%, accuracy@5=53.22%
Find RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth exist, delete is at first before saving
save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/checkpoint/seed-0-basic.pth
Find RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth exist, delete is at first before saving
save checkpoint into RANDOM_LoW_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth
|none~0|+|none~0|nor_conv_1x1~1|+|skip_connect~0|nor_conv_1x1~1|avg_pool_3x3~2|
datasets : ['cifar10-valid', 'cifar10', 'cifar100', 'ImageNet16-120'], extra-info : arch-index=13321
cifar10-valid  FLOP= 15.65 M, Params=0.129 MB, latency=16.12 ms.
cifar10-valid  train : [loss = 0.272, top1 = 90.94%], valid : [loss = 0.465, top1 = 84.68%]
cifar10        FLOP= 15.65 M, Params=0.129 MB, latency=15.34 ms.
cifar10        train : [loss = 0.300, top1 = 89.

...

final evaluation [00/100] : (Structure(4 nodes with |none~0|+|nor_conv_3x3~0|nor_conv_1x1~1|+|skip_connect~0|nor_conv_1x1~1|nor_conv_1x1~2|), [(('none', 0),), (('nor_conv_3x3', 0), ('nor_conv_1x1', 1)), (('skip_connect', 0), ('nor_conv_1x1', 1), ('nor_conv_1x1', 2))], ['none', 'nor_conv_3x3', 'nor_conv_1x1', 'skip_connect', 'nor_conv_1x1', 'nor_conv_1x1']) : accuracy=16.42%, loss=161.244
final evaluation [01/100] : (Structure(4 nodes with |nor_conv_3x3~0|+|avg_pool_3x3~0|none~1|+|nor_conv_1x1~0|nor_conv_1x1~1|avg_pool_3x3~2|), [(('nor_conv_3x3', 0),), (('avg_pool_3x3', 0), ('none', 1)), (('nor_conv_1x1', 0), ('nor_conv_1x1', 1), ('avg_pool_3x3', 2))], ['nor_conv_3x3', 'avg_pool_3x3', 'none', 'nor_conv_1x1', 'nor_conv_1x1', 'avg_pool_3x3']) : accuracy=16.30%, loss=261.358
final evaluation [02/100] : (Structure(4 nodes with |skip_connect~0|+|nor_conv_1x1~0|skip_connect~1|+|none~0|nor_conv_1x1~1|skip_connect~2|), [(('skip_connect', 0),), (('nor_conv_1x1', 0), ('skip_connect', 1)), (('none

final evaluation [22/100] : (Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|skip_connect~1|+|none~0|avg_pool_3x3~1|avg_pool_3x3~2|), [(('skip_connect', 0),), (('nor_conv_3x3', 0), ('skip_connect', 1)), (('none', 0), ('avg_pool_3x3', 1), ('avg_pool_3x3', 2))], ['skip_connect', 'nor_conv_3x3', 'skip_connect', 'none', 'avg_pool_3x3', 'avg_pool_3x3']) : accuracy=16.36%, loss=116.298
final evaluation [23/100] : (Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_1x1~0|none~1|+|avg_pool_3x3~0|none~1|nor_conv_3x3~2|), [(('nor_conv_3x3', 0),), (('nor_conv_1x1', 0), ('none', 1)), (('avg_pool_3x3', 0), ('none', 1), ('nor_conv_3x3', 2))], ['nor_conv_3x3', 'nor_conv_1x1', 'none', 'avg_pool_3x3', 'none', 'nor_conv_3x3']) : accuracy=17.63%, loss=264.441
final evaluation [24/100] : (Structure(4 nodes with |none~0|+|avg_pool_3x3~0|nor_conv_1x1~1|+|avg_pool_3x3~0|avg_pool_3x3~1|skip_connect~2|), [(('none', 0),), (('avg_pool_3x3', 0), ('nor_conv_1x1', 1)), (('avg_pool_3x3', 0), ('avg_pool_3x3', 

final evaluation [44/100] : (Structure(4 nodes with |skip_connect~0|+|skip_connect~0|nor_conv_1x1~1|+|nor_conv_3x3~0|avg_pool_3x3~1|nor_conv_3x3~2|), [(('skip_connect', 0),), (('skip_connect', 0), ('nor_conv_1x1', 1)), (('nor_conv_3x3', 0), ('avg_pool_3x3', 1), ('nor_conv_3x3', 2))], ['skip_connect', 'skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3', 'nor_conv_3x3']) : accuracy=16.51%, loss=165.905
final evaluation [45/100] : (Structure(4 nodes with |avg_pool_3x3~0|+|none~0|avg_pool_3x3~1|+|none~0|avg_pool_3x3~1|nor_conv_1x1~2|), [(('avg_pool_3x3', 0),), (('none', 0), ('avg_pool_3x3', 1)), (('none', 0), ('avg_pool_3x3', 1), ('nor_conv_1x1', 2))], ['avg_pool_3x3', 'none', 'avg_pool_3x3', 'none', 'avg_pool_3x3', 'nor_conv_1x1']) : accuracy=16.87%, loss=32.874
final evaluation [46/100] : (Structure(4 nodes with |skip_connect~0|+|none~0|nor_conv_3x3~1|+|skip_connect~0|avg_pool_3x3~1|nor_conv_3x3~2|), [(('skip_connect', 0),), (('none', 0), ('nor_conv_3x3', 1)), (('skip_connect'

final evaluation [66/100] : (Structure(4 nodes with |skip_connect~0|+|nor_conv_1x1~0|nor_conv_3x3~1|+|nor_conv_3x3~0|avg_pool_3x3~1|skip_connect~2|), [(('skip_connect', 0),), (('nor_conv_1x1', 0), ('nor_conv_3x3', 1)), (('nor_conv_3x3', 0), ('avg_pool_3x3', 1), ('skip_connect', 2))], ['skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'nor_conv_3x3', 'avg_pool_3x3', 'skip_connect']) : accuracy=16.54%, loss=186.481
final evaluation [67/100] : (Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|skip_connect~1|+|skip_connect~0|none~1|nor_conv_1x1~2|), [(('skip_connect', 0),), (('nor_conv_3x3', 0), ('skip_connect', 1)), (('skip_connect', 0), ('none', 1), ('nor_conv_1x1', 2))], ['skip_connect', 'nor_conv_3x3', 'skip_connect', 'skip_connect', 'none', 'nor_conv_1x1']) : accuracy=16.36%, loss=284.735
final evaluation [68/100] : (Structure(4 nodes with |skip_connect~0|+|avg_pool_3x3~0|avg_pool_3x3~1|+|skip_connect~0|none~1|skip_connect~2|), [(('skip_connect', 0),), (('avg_pool_3x3', 0), ('avg

final evaluation [88/100] : (Structure(4 nodes with |skip_connect~0|+|skip_connect~0|nor_conv_1x1~1|+|skip_connect~0|nor_conv_1x1~1|nor_conv_1x1~2|), [(('skip_connect', 0),), (('skip_connect', 0), ('nor_conv_1x1', 1)), (('skip_connect', 0), ('nor_conv_1x1', 1), ('nor_conv_1x1', 2))], ['skip_connect', 'skip_connect', 'nor_conv_1x1', 'skip_connect', 'nor_conv_1x1', 'nor_conv_1x1']) : accuracy=15.89%, loss=188.234
final evaluation [89/100] : (Structure(4 nodes with |none~0|+|skip_connect~0|avg_pool_3x3~1|+|nor_conv_3x3~0|none~1|nor_conv_1x1~2|), [(('none', 0),), (('skip_connect', 0), ('avg_pool_3x3', 1)), (('nor_conv_3x3', 0), ('none', 1), ('nor_conv_1x1', 2))], ['none', 'skip_connect', 'avg_pool_3x3', 'nor_conv_3x3', 'none', 'nor_conv_1x1']) : accuracy=16.97%, loss=268.763
final evaluation [90/100] : (Structure(4 nodes with |nor_conv_3x3~0|+|skip_connect~0|avg_pool_3x3~1|+|nor_conv_1x1~0|none~1|nor_conv_3x3~2|), [(('nor_conv_3x3', 0),), (('skip_connect', 0), ('avg_pool_3x3', 1)), (('nor_